In [2]:
import wandb
import numpy as np
import os
import random
import glob
import pandas as pd
import time
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier, VotingClassifier
from sklearn.datasets import fetch_covtype
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.io import arff
from sklearn.datasets import fetch_openml
from joblib import Parallel, delayed

In [3]:
wandb.login()

wandb: Currently logged in as: brinashong. Use `wandb login --relogin` to force relogin


True

In [4]:
%run 'common.ipynb'

In [5]:
# Shared variables
OUTPUT_FOLDER = 'csv_files'
FEATURE_FOLDER = 'features'
TEST_COUNT = 500

In [6]:
%run kdd.ipynb

In [7]:
# Should already be one hot encoded and label encoded
# all_df, main_labels, config = get_processed_covtype_df()
# all_df, main_labels, config = get_processed_cicids_df()
all_df, main_labels, config = get_processed_kdd_df()
TARGET_COLUMN = config['TARGET_COLUMN']
NORMAL_TARGET = config['NORMAL_TARGET']
TARGET_DICT = config['TARGET_DICT']
INV_TARGET_DICT = config['INV_TARGET_DICT']
NUMERICAL_COLUMNS = config['NUMERICAL_COLUMNS']
CATEGORICAL_COLUMNS = config['CATEGORICAL_COLUMNS']
ORDINAL_COLUMNS = config['ORDINAL_COLUMNS']
all_df.head()

Normal class:  0    smurf.
dtype: object
1 17
TARGET_DICT {0: 'apache2', 1: 'back', 2: 'buffer_overflow', 3: 'ftp_write', 4: 'guess_passwd', 5: 'httptunnel', 6: 'imap', 7: 'ipsweep', 8: 'land', 9: 'loadmodule', 10: 'mailbomb', 11: 'mscan', 12: 'multihop', 13: 'named', 14: 'neptune', 15: 'nmap', 16: 'normal', 17: 'perl', 18: 'phf', 19: 'pod', 20: 'portsweep', 21: 'processtable', 22: 'ps', 23: 'rootkit', 24: 'saint', 25: 'satan', 26: 'sendmail', 27: 'smurf', 28: 'snmpgetattack', 29: 'snmpguess', 30: 'sqlattack', 31: 'teardrop', 32: 'udpstorm', 33: 'warezmaster', 34: 'worm', 35: 'xlock', 36: 'xsnoop', 37: 'xterm'}
2 17
NORMAL_TARGET 16
['duration', 'protocol_type_icmp', 'protocol_type_tcp', 'protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,16
1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,16
2,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,16
3,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,28
4,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,28


In [8]:
all_df.shape

(311029, 118)

In [9]:
# Create a folder to save the CSVs
remove_files_from_directory(OUTPUT_FOLDER)
remove_files_from_directory(FEATURE_FOLDER)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(FEATURE_FOLDER, exist_ok=True)

All files in csv_files have been removed.
All files in features have been removed.


In [10]:
# Save all data as csv
all_df.to_csv('all_data.csv' ,index = False)

In [13]:
# Get X and y from all_df
X_df = all_df.drop(columns=[TARGET_COLUMN])
y_df = all_df[TARGET_COLUMN]

# Split the data into training and test sets
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
print(X_train_df.shape, X_test_df.shape, y_train_df.shape, y_test_df.shape)

(248823, 117) (62206, 117) (248823,) (62206,)


In [14]:
# Concatenate X_train and y_train along the columns (axis=1)
X_y_train_df = pd.concat([X_train_df, y_train_df], axis=1)
X_y_train_df.head()

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
147391,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,231,1.00,0.00,0.02,0.01,0.0,0.0,0.0,0.0,16
107622,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17,0.07,0.06,0.00,0.00,0.0,0.0,1.0,1.0,14
53448,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,27
3099,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,27
208489,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,27


In [15]:
# Initialize the standard scaler
scaler = StandardScaler()

# Fit and transform the numeric columns
X_train_scaled_df = X_train_df.copy()
X_train_scaled_df[NUMERICAL_COLUMNS] = scaler.fit_transform(X_train_scaled_df[NUMERICAL_COLUMNS])
# print('X_train_scaled_df', X_train_scaled_df)

# Use the same scaler to transform X_test
X_test_scaled_df = X_test_df.copy()
X_test_scaled_df[NUMERICAL_COLUMNS] = scaler.transform(X_test_scaled_df[NUMERICAL_COLUMNS])
# print('X_test_scaled_df', X_test_scaled_df)

In [22]:
# Loop through each cover type and create a dataset
benign = 2359289
abnormal_type_counts = all_df[TARGET_COLUMN].value_counts()
abnormal_type_dict = abnormal_type_counts.to_dict()
print('abnormal_type_dict', abnormal_type_dict)

benign_ratio = 2  # Adjust this ratio as needed
min_benign_samples = 50  # Ensure at least 50 benign samples are always included

for label, name in TARGET_DICT.items():
    if label == NORMAL_TARGET:
        continue  # Skip the normal target
    
    a, b = 0, 0  # Track abnormal and benign sample counts
    
    # Open the output file for writing
    output_path = os.path.join(OUTPUT_FOLDER, f"{name}.csv")
    with open(output_path, "w") as ths:
        ths.write(','.join(main_labels) + "\n")
        
        # Calculate the number of benign samples based on the fixed ratio
        abnormal_count = abnormal_type_dict[label]
        benign_num = max(min(int(abnormal_count * benign_ratio), benign), min_benign_samples)
        # print(f'Creating {name}.csv with {benign_num} benign samples and {abnormal_count} abnormal samples.')

        # Collect normal (benign) rows and abnormal rows
        benign_rows = []
        abnormal_rows = []

        # Read all_data.csv line by line and collect rows
        with open("all_data.csv", "r") as file:
            for i, line in enumerate(file):
                if i == 0:
                    continue  # Skip the header row
                k = line.strip().split(",")  # Strip newline and split the line
                
                # Collect normal rows
                if int(k[-1]) == NORMAL_TARGET:
                    benign_rows.append(line)
                
                # Collect abnormal rows that match the current label
                elif int(k[-1]) == label:
                    abnormal_rows.append(line)

        # Randomly sample benign rows
        if len(benign_rows) > benign_num:
            benign_rows = random.sample(benign_rows, benign_num)
        else:
            benign_rows = random.sample(benign_rows, len(benign_rows))  # Shuffle if fewer than required

        # Concatenate benign and abnormal rows
        combined_rows = benign_rows + abnormal_rows
        
        # Shuffle the combined rows
        random.shuffle(combined_rows)

        # Write the shuffled rows to the output file
        for row in combined_rows:
            ths.write(row)

        # Print number of rows written
        b = len(benign_rows)
        a = len(abnormal_rows)
        print(f"{name}.csv created with {a + b} rows. ({b} benign and {a} abnormal rows)")

print("All datasets created successfully!")

abnormal_type_dict {27: 164091, 16: 60593, 14: 58001, 28: 7741, 10: 5000, 4: 4367, 29: 2406, 25: 1633, 33: 1602, 1: 1098, 11: 1053, 0: 794, 21: 759, 24: 736, 20: 354, 7: 306, 5: 158, 19: 87, 15: 84, 2: 22, 12: 18, 13: 17, 26: 17, 22: 16, 23: 13, 37: 13, 31: 12, 35: 9, 8: 9, 36: 4, 3: 3, 9: 2, 17: 2, 32: 2, 34: 2, 18: 2, 30: 2, 6: 1}
apache2.csv created with 2382 rows. (1588 benign and 794 abnormal rows)
back.csv created with 3294 rows. (2196 benign and 1098 abnormal rows)
buffer_overflow.csv created with 72 rows. (50 benign and 22 abnormal rows)
ftp_write.csv created with 53 rows. (50 benign and 3 abnormal rows)
guess_passwd.csv created with 13101 rows. (8734 benign and 4367 abnormal rows)
httptunnel.csv created with 474 rows. (316 benign and 158 abnormal rows)
imap.csv created with 51 rows. (50 benign and 1 abnormal rows)
ipsweep.csv created with 918 rows. (612 benign and 306 abnormal rows)
land.csv created with 59 rows. (50 benign and 9 abnormal rows)
loadmodule.csv created with 52 r

In [25]:
# tdf = pd.read_csv('csv_files/imap.csv')
# tdf['attack'].unique()
show_missing_values(

array([16,  6])

In [23]:
seconds = time.time()

# CSV files names:
csv_files=os.listdir(OUTPUT_FOLDER)# It creates a list of file names in the "attacks" folder.
print('csv_files',csv_files)

ths = open("importance_list.csv", "w")
feature_importances = {}
ensemble_models = {}
SVMs = {}

def train_individual_model(model, X, y):
    model.fit(X, y)
    return model
    
def train_models(csv_file):
    print('csv file', csv_file)
    
    X_df, y_df, df = get_anomaly_X_y_from_csv(csv_file, main_labels, TARGET_COLUMN, NORMAL_TARGET, OUTPUT_FOLDER)

    #computing the feature importances
    forest = sk.ensemble.RandomForestRegressor(n_estimators=250,random_state=0)
    forest.fit(X_df, y_df)
    importances = forest.feature_importances_
    label = csv_file.split(".")[0]
    print('importances', importances, label)
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]
    refclasscol=list(df.columns.values)
    impor_bars = pd.DataFrame({'Features':refclasscol[0:20],'importance':importances[0:20]})
    impor_bars = impor_bars.sort_values('importance',ascending=False)
    important_features = impor_bars['Features'].to_list()[:5]
    impor_bars = impor_bars.set_index('Features')
    print('important_features', important_features)
    feature_importances[label] = important_features

    X_scaled_df = X_df.copy()
    X_scaled_df[NUMERICAL_COLUMNS] = scaler.transform(X_scaled_df[NUMERICAL_COLUMNS])

    svm = SVC()
    knn = KNeighborsClassifier(n_neighbors=5)
    decision_tree = DecisionTreeClassifier()
    random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
    logistic_regression = LogisticRegression(max_iter=1000)
    gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
    
    X_train_class = df.iloc[:, df.columns.get_indexer(important_features)]
    X_train_class_scaled = X_scaled_df.iloc[:, df.columns.get_indexer(important_features)]
    # print('X_train_class', X_train_class)
    y_train_class = y_df
    # print('y_train_class', y_train_class)
    if len(y_train_class) > 0:  # Ensure there are samples for this class
        scaled_models = [svm, knn, logistic_regression]
        non_scaled_models = [decision_tree, random_forest, gradient_boosting]
        
        # Train scaled models
        scaled_models = Parallel(n_jobs=-1)(
            delayed(train_individual_model)(model, X_train_class_scaled, y_train_class)
            for model in scaled_models
        )
        
        # Train non-scaled models
        non_scaled_models = Parallel(n_jobs=-1)(
            delayed(train_individual_model)(model, X_train_class, y_train_class)
            for model in non_scaled_models
        )
        
        # Combine trained models
        svm, knn, logistic_regression = scaled_models
        decision_tree, random_forest, gradient_boosting = non_scaled_models
    else:
        print(f'no data for {label}')
        
    SVMs[label] = svm

    # Create an ensemble from the classifiers using VotingClassifier
    voting_clf = VotingClassifier(estimators=[
        ('svm', svm),
        ('knn', knn),
        ('dt', decision_tree),
        ('rf', random_forest),
        ('lr', logistic_regression),
        ('gb', gradient_boosting)
    ], voting='hard')  # Use 'soft' for probability-based voting
    voting_clf.fit(X_train_class_scaled, y_train_class)
    ensemble_models[label] = voting_clf
    
    plt.rcParams['figure.figsize'] = (10, 5)
    impor_bars.plot.bar();
    # #printing the feature importances  
    count=0
    fea_ture=label+"=["
    for i in impor_bars.index:
        fea_ture=fea_ture+"\""+str(i)+"\","
        count+=1
        if count==5:
            fea_ture=fea_ture[0:-1]+"]"
            break     
    print(label,"importance list:")
    print(label,"\n",impor_bars.head(20),"\n\n\n")
    print(fea_ture)
    plt.title(label+" Cover type - Feature Importance")
    plt.ylabel('Importance')
    plt.savefig(os.path.join(FEATURE_FOLDER, label+".pdf"),bbox_inches='tight', format = 'pdf')
    # ths.write((  fea_ture ) )
    plt.tight_layout()
    plt.show()
    print("-----------------------------------------------------------------------------------------------\n\n\n\n")

    return feature_importances, ensemble_models, SVMs

# Parallelize the training across multiple CSV files
results = Parallel(n_jobs=-1)(delayed(train_models)(csv_file) for csv_file in csv_files)

# After parallel execution, collect results
for feature_importances_res, ensemble_models_res, SVMs_res in results:
    feature_importances.update(feature_importances_res)
    ensemble_models.update(ensemble_models_res)
    SVMs.update(SVMs_res)

# Write feature importances to a CSV file after parallel processing
# with open("importance_list.csv", "w") as ths:
#     for label, features in feature_importances.items():
#         fea_ture = f"{label}=[{','.join(f'\"{feat}\"' for feat in features)}]\n"
#         ths.write(fea_ture)
  
print('feature_importances', feature_importances)
print("Total operation time: = ", time.time() - seconds, "seconds")
ths.close()

csv_files ['named.csv', 'snmpgetattack.csv', 'ipsweep.csv', 'httptunnel.csv', 'sendmail.csv', 'guess_passwd.csv', 'satan.csv', 'ps.csv', 'ftp_write.csv', 'xlock.csv', 'udpstorm.csv', 'teardrop.csv', 'phf.csv', 'pod.csv', 'perl.csv', 'nmap.csv', 'portsweep.csv', 'saint.csv', 'rootkit.csv', 'warezmaster.csv', 'xsnoop.csv', 'sqlattack.csv', 'processtable.csv', 'snmpguess.csv', 'mailbomb.csv', 'imap.csv', 'neptune.csv', 'xterm.csv', 'mscan.csv', 'buffer_overflow.csv', 'loadmodule.csv', 'worm.csv', 'multihop.csv', 'back.csv', 'apache2.csv', '.ipynb_checkpoints', 'land.csv', 'smurf.csv']


ValueError: The number of classes has to be greater than one; got 1 class

In [21]:
seconds = time.time()

# CSV files names:
csv_files=os.listdir(OUTPUT_FOLDER)# It creates a list of file names in the "attacks" folder.
print('csv_files',csv_files)

ths = open("importance_list.csv", "w")
feature_importances = {}
ensemble_models = {}
SVMs = {}

def train_models(csv_file):
    print('csv file', csv_file)
    
    X_df, y_df, df = get_anomaly_X_y_from_csv(csv_file, main_labels, TARGET_COLUMN, NORMAL_TARGET, OUTPUT_FOLDER)

    #computing the feature importances
    forest = sk.ensemble.RandomForestRegressor(n_estimators=250,random_state=0)
    forest.fit(X_df, y_df)
    importances = forest.feature_importances_
    label = csv_file.split(".")[0]
    print('importances', importances, label)
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]
    refclasscol=list(df.columns.values)
    impor_bars = pd.DataFrame({'Features':refclasscol[0:20],'importance':importances[0:20]})
    impor_bars = impor_bars.sort_values('importance',ascending=False)
    important_features = impor_bars['Features'].to_list()[:5]
    impor_bars = impor_bars.set_index('Features')
    print('important_features', important_features)
    feature_importances[label] = important_features

    X_scaled_df = X_df.copy()
    X_scaled_df[NUMERICAL_COLUMNS] = scaler.transform(X_scaled_df[NUMERICAL_COLUMNS])

    svm = SVC()
    knn = KNeighborsClassifier(n_neighbors=5)
    decision_tree = DecisionTreeClassifier()
    random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
    logistic_regression = LogisticRegression(max_iter=1000)
    gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
    
    X_train_class = df.iloc[:, df.columns.get_indexer(important_features)]
    X_train_class_scaled = X_scaled_df.iloc[:, df.columns.get_indexer(important_features)]
    # print('X_train_class', X_train_class)
    y_train_class = y_df
    # print('y_train_class', y_train_class)
    if len(y_train_class) > 0:  # Ensure there are samples for this class
        svm.fit(X_train_class_scaled, y_train_class)
        knn.fit(X_train_class_scaled, y_train_class)
        decision_tree.fit(X_train_class, y_train_class)
        random_forest.fit(X_train_class, y_train_class)
        logistic_regression.fit(X_train_class_scaled, y_train_class)
        gradient_boosting.fit(X_train_class, y_train_class)
    else:
        print(f'no data for {label}')
    SVMs[label] = svm

    # Create an ensemble from the classifiers using VotingClassifier
    voting_clf = VotingClassifier(estimators=[
        ('svm', svm),
        ('knn', knn),
        ('dt', decision_tree),
        ('rf', random_forest),
        ('lr', logistic_regression),
        ('gb', gradient_boosting)
    ], voting='hard')  # Use 'soft' for probability-based voting
    voting_clf.fit(X_train_class_scaled, y_train_class)
    ensemble_models[label] = voting_clf
    
    plt.rcParams['figure.figsize'] = (10, 5)
    impor_bars.plot.bar();
    # #printing the feature importances  
    count=0
    fea_ture=label+"=["
    for i in impor_bars.index:
        fea_ture=fea_ture+"\""+str(i)+"\","
        count+=1
        if count==5:
            fea_ture=fea_ture[0:-1]+"]"
            break     
    print(label,"importance list:")
    print(label,"\n",impor_bars.head(20),"\n\n\n")
    print(fea_ture)
    plt.title(label+" Cover type - Feature Importance")
    plt.ylabel('Importance')
    plt.savefig(os.path.join(FEATURE_FOLDER, label+".pdf"),bbox_inches='tight', format = 'pdf')
    # ths.write((  fea_ture ) )
    plt.tight_layout()
    plt.show()
    print("-----------------------------------------------------------------------------------------------\n\n\n\n")

    return feature_importances, ensemble_models, SVMs

# Parallelize the training across multiple CSV files
results = Parallel(n_jobs=-1)(delayed(train_models)(csv_file) for csv_file in csv_files)

# After parallel execution, collect results
for feature_importances_res, ensemble_models_res, SVMs_res in results:
    feature_importances.update(feature_importances_res)
    ensemble_models.update(ensemble_models_res)
    SVMs.update(SVMs_res)

# Write feature importances to a CSV file after parallel processing
# with open("importance_list.csv", "w") as ths:
#     for label, features in feature_importances.items():
#         fea_ture = f"{label}=[{','.join(f'\"{feat}\"' for feat in features)}]\n"
#         ths.write(fea_ture)
  
print('feature_importances', feature_importances)
print("Total operation time: = ", time.time() - seconds, "seconds")
ths.close()

csv_files ['named.csv', 'snmpgetattack.csv', 'ipsweep.csv', 'httptunnel.csv', 'sendmail.csv', 'guess_passwd.csv', 'satan.csv', 'ps.csv', 'ftp_write.csv', 'xlock.csv', 'udpstorm.csv', 'teardrop.csv', 'phf.csv', 'pod.csv', 'perl.csv', 'nmap.csv', 'portsweep.csv', 'saint.csv', 'rootkit.csv', 'warezmaster.csv', 'xsnoop.csv', 'sqlattack.csv', 'processtable.csv', 'snmpguess.csv', 'mailbomb.csv', 'imap.csv', 'neptune.csv', 'xterm.csv', 'mscan.csv', 'buffer_overflow.csv', 'loadmodule.csv', 'worm.csv', 'multihop.csv', 'back.csv', 'apache2.csv', '.ipynb_checkpoints', 'land.csv', 'smurf.csv']


ValueError: The number of classes has to be greater than one; got 1 class

In [14]:
# Initialize variables to track the best model and best performance
best_k = None
best_score = -1
best_model = None

seconds = time.time()

for k in range(3, 21, 2):
    wandb.init(project="pipeline-knn-kdd-dataset", name=str(k)+" neighbours")
    print("k: ", k)
    
    # Step 1: Train KNN to classify
    # knn = KNeighborsClassifier(n_neighbors=k)
    # knn.fit(X_train_scaled_df, y_train_df)
    # knn.fit(X_train_df, y_train_df)
    
    # Step 2: Predict classes for the test set
    # X_test_df = X_test_df[:TEST_COUNT]
    # X_test_scaled_df = X_test_scaled_df[:TEST_COUNT]
    # y_test_df = y_test_df[:TEST_COUNT]

    # Step 1: Initialize KNN classifier with current k
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Step 2: Perform 5-fold cross-validation on the training set
    cv_scores = cross_val_score(knn, X_train_df, y_train_df, cv=5, scoring='accuracy')
    avg_cv_score = cv_scores.mean()  # Get the average cross-validation score
    print(f"Cross-validation score for k={k}: {avg_cv_score}")
    
    # knn_predictions = knn.predict(X_test_scaled)
    # predictions = knn.predict(X_test_df)
    # print('knn_predictions', predictions)
    
    op_time = time.time() - seconds
    print("Total operation time: = ", op_time, "seconds")
    
    # conf_matrix, class_report, acc_score = evaluate(y_test_df, predictions, heading='-----KNN Evaluation-----')
    # wandb_log(conf_matrix, class_report, acc_score)
    
    wandb.log({
        "Cross-validation Score": avg_cv_score,
        "Total operation time": op_time
    })

     # Step 3: Track the best `k` based on the highest cross-validation score
    if avg_cv_score > best_score:
        best_score = avg_cv_score
        best_k = k
        best_model = knn  # Save the KNN model with the best `k`

    wandb.finish()

# After the loop: Train the best KNN model on the entire training set
print(f"The best k is {best_k} with a cross-validation score of {best_score}")
best_model.fit(X_train_df, y_train_df)

# Step 4: Evaluate the best model on the test set
X_test_df = X_test_df[:TEST_COUNT]
y_test_df = y_test_df[:TEST_COUNT]

# Make predictions on the test set
knn_predictions = best_model.predict(X_test_df)
print("Final predictions using the best model:", knn_predictions)

# Evaluate the performance on the test set
conf_matrix, class_report, acc_score = evaluate(y_test_df, knn_predictions, heading="Best KNN Model Evaluation")
print(f"Accuracy on test set with best k={best_k}: {acc_score}")

# Log the final test results to W&B
wandb.init(project="pipeline-knn-kdd-dataset", name="best_model_evaluation")
wandb_log(conf_matrix, class_report, acc_score)
wandb.finish()

k:  3
Cross-validation score for k=3: 0.9658289756892552
Total operation time: =  148.28774857521057 seconds


Cross-validation Score,▁
Total operation time,▁
Cross-validation Score,0.96583
Total operation time,148.28775


The best k is 3 with a cross-validation score of 0.9658289756892552
Final predictions using the best model: [1 2 2 2 2 3 2 1 2 2 2 1 5 2 2 1 2 5 3 2 1 2 2 1 6 1 1 2 1 2 1 2 2 3 1 2 2
 2 2 1 2 1 1 2 2 1 2 2 7 2 1 1 6 2 1 1 7 2 2 2 2 1 1 2 2 2 3 2 2 1 5 1 2 3
 1 3 2 1 1 2 2 1 2 2 2 1 1 3 1 2 1 1 2 1 3 3 2 2 1 1 6 6 2 2 1 2 5 6 6 1 2
 1 2 1 1 1 3 1 2 3 1 1 1 1 2 2 1 1 1 2 1 2 2 2 5 1 1 1 2 3 1 2 2 3 1 1 1 2
 2 2 1 7 2 2 1 1 2 2 1 2 1 3 1 2 2 2 2 2 2 2 2 1 2 1 1 1 1 2 2 2 1 2 2 2 2
 2 1 2 2 1 1 1 2 2 1 3 3 2 1 1 2 7 1 1 1 1 2 2 2 3 2 5 3 3 2 1 2 1 2 2 3 1
 2 2 3 2 2 2 1 1 2 5 1 1 1 2 2 1 2 2 1 1 2 1 1 6 2 1 2 7 2 1 3 5 1 2 3 2 2
 1 2 2 1 2 2 2 2 1 1 2 1 2 1 1 2 6 1 3 2 7 1 1 2 2 2 2 2 2 7 1 1 1 1 1 2 2
 5 2 2 2 1 6 2 1 1 2 7 1 2 3 2 2 2 2 2 6 1 5 2 2 1 6 2 2 1 1 1 1 2 2 2 7 3
 3 2 1 2 2 2 3 2 2 1 2 1 1 2 1 7 2 1 3 3 5 2 2 1 3 3 2 2 1 2 2 2 2 2 2 1 2
 2 1 2 2 1 2 1 1 2 2 2 1 2 1 3 1 3 2 2 2 6 2 3 2 2 1 2 2 6 1 2 1 2 2 2 2 1
 2 2 7 2 2 3 1 2 1 1 2 2 2 1 7 1 2 2 1 6 2 2 1 1 2 2 2 6 2 2 5 1 6 

/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Accuracy Score,▁
f1-score_avg,▁
precision_avg,▁
recall_avg,▁
Accuracy Score,0.984
f1-score_avg,0.98297
precision_avg,0.98232
recall_avg,0.984


csv file Spruce-Fir.csv
importances [3.27792519e-01 2.79286653e-02 1.63415314e-02 6.75524954e-02
 4.69016726e-02 1.58180294e-01 2.81521628e-02 3.90647018e-02
 2.19325803e-02 1.57215058e-01 8.00204582e-03 5.00552027e-03
 1.04513981e-02 1.67397951e-07 0.00000000e+00 1.42215969e-05
 1.47483326e-05 1.93429342e-04 0.00000000e+00 0.00000000e+00
 6.95393215e-07 9.52867953e-05 1.83302616e-04 8.15366337e-04
 3.42573691e-04 1.53201802e-03 3.25231064e-03 0.00000000e+00
 0.00000000e+00 1.18171505e-03 2.42002225e-04 4.89926975e-06
 1.24509056e-03 3.00706212e-03 7.68923201e-04 1.23347759e-02
 1.08121855e-02 7.57167704e-03 1.39963583e-05 1.38922156e-04
 1.55518227e-03 1.18679619e-04 1.02146627e-02 2.72535698e-03
 6.58068874e-03 1.11305597e-02 6.67661334e-03 7.52689844e-04
 5.81856816e-06 1.06369486e-04 0.00000000e+00 6.02698132e-04
 7.64023417e-04 4.80643156e-04] Spruce-Fir
important_features ['Elevation', 'Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_T

In [20]:
print(len(X_test_scaled_df))

116203


In [15]:
print(SVMs)

{'Krummholz': SVC(), 'CottonwoodWillow': SVC(), 'Spruce-Fir': SVC(), 'Aspen': SVC(), 'DouglasFir': SVC(), 'PonderosaPine': SVC()}


In [21]:
svm_predictions = []
seconds = time.time()

X_test_df = X_test_df[:TEST_COUNT]
X_test_scaled_df = X_test_scaled_df[:TEST_COUNT]
y_test_df = y_test_df[:TEST_COUNT]

for i in range(len(X_test_scaled_df)):
    test_instance_df = X_test_scaled_df.iloc[i].to_frame().T
    if knn_predictions[i] == NORMAL_TARGET:
        svm_predictions.append((0, NORMAL_TARGET, y_test_df.iloc[i]))
        continue
    predicted_label = TARGET_DICT[knn_predictions[i]]
    print('predicted_label', predicted_label)
    selected_features = feature_importances[predicted_label]
    # print('selected_features', selected_features)
    svm_model = SVMs[predicted_label]
    
    # Select features for the SVM
    test_instance_selected_df = test_instance_df[selected_features]
    svm_prediction = svm_model.predict(test_instance_selected_df)
    print('svm prediction: ', svm_prediction, 'knn prediction: ', INV_TARGET_DICT[predicted_label], 'actual value: ', y_test_df.iloc[i])
    svm_predictions.append((svm_prediction, INV_TARGET_DICT[predicted_label], y_test_df.iloc[i]))

print("Total operation time: = ", time.time() - seconds, "seconds")

predicted_label Spruce-Fir
svm prediction:  [0] knn prediction:  1 actual value:  1
predicted_label PonderosaPine
svm prediction:  [0] knn prediction:  3 actual value:  3
predicted_label Spruce-Fir
svm prediction:  [0] knn prediction:  1 actual value:  1
predicted_label Spruce-Fir
svm prediction:  [0] knn prediction:  1 actual value:  1
predicted_label Aspen
svm prediction:  [0] knn prediction:  5 actual value:  5
predicted_label Spruce-Fir
svm prediction:  [1] knn prediction:  1 actual value:  1
predicted_label Aspen
svm prediction:  [1] knn prediction:  5 actual value:  5
predicted_label PonderosaPine
svm prediction:  [0] knn prediction:  3 actual value:  3
predicted_label Spruce-Fir
svm prediction:  [0] knn prediction:  1 actual value:  1
predicted_label Spruce-Fir
svm prediction:  [0] knn prediction:  1 actual value:  1
predicted_label DouglasFir
svm prediction:  [0] knn prediction:  6 actual value:  6
predicted_label Spruce-Fir
svm prediction:  [0] knn prediction:  1 actual value:

In [22]:
wandb.init(project="pipeline-kdd-dataset", name="anomalies-svm-only")

# Now evaluate SVM predictions only for the anomalies detected by KNN
# Create a mask for test instances that KNN classified as anomalies
anomaly_mask = knn_predictions != NORMAL_TARGET  # Assuming normal_target is your normal class
print('anomaly_mask', len(anomaly_mask))

# Get true labels and predictions for anomalies
svm_predictions_actual = [t[1] for t in svm_predictions]
print(svm_predictions_actual)
# svm_predictions_actual = np.array(svm_predictions_actual)
svm_predictions_actual_df = pd.DataFrame(svm_predictions_actual, columns=[TARGET_COLUMN])
y_test_anomalies_df = y_test_df[anomaly_mask]
svm_predictions_anomalies_df = svm_predictions_actual_df[anomaly_mask]
print('Lengths: ', len(y_test_anomalies_df), len(svm_predictions_anomalies_df))

# Evaluate SVM only on the anomalies
conf_matrix, class_report, acc_score = evaluate(y_test_anomalies_df, svm_predictions_anomalies_df, heading='SVM Evaluation (for anomalies)')
wandb_log(conf_matrix, class_report, acc_score)
wandb.finish()

# Evaluate SVM on all test data
wandb.init(project="pipeline-kdd-dataset", name="overall-svm-only")
conf_matrix, class_report, acc_score = evaluate(y_test_df, svm_predictions_actual_df, heading='SVM Evaluation (for all)')
wandb_log(conf_matrix, class_report, acc_score)
wandb.finish()

anomaly_mask 500
[1, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 1, 5, 2, 2, 1, 2, 5, 3, 2, 1, 2, 2, 1, 6, 1, 1, 2, 1, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 7, 2, 1, 1, 6, 2, 1, 1, 7, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 2, 2, 1, 5, 1, 2, 3, 1, 3, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 3, 1, 2, 1, 1, 2, 1, 3, 3, 2, 2, 1, 1, 6, 6, 2, 2, 1, 2, 5, 6, 6, 1, 2, 1, 2, 1, 1, 1, 3, 1, 2, 3, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 5, 1, 1, 1, 2, 3, 1, 2, 2, 3, 1, 1, 1, 2, 2, 2, 1, 7, 2, 2, 1, 1, 2, 2, 1, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 3, 3, 2, 1, 1, 2, 7, 1, 1, 1, 1, 2, 2, 2, 3, 2, 5, 3, 3, 2, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 2, 2, 1, 1, 2, 5, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 6, 2, 1, 2, 7, 2, 1, 3, 5, 1, 2, 3, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 6, 1, 3, 2, 7, 1, 1, 2, 2, 2, 2, 2, 2, 7, 1, 1, 1, 1, 1, 2, 2, 5, 2, 2, 2, 1, 6, 2, 1, 1, 2, 7, 1, 2, 3, 2, 2, 2, 2, 2, 6, 1, 5, 2, 2, 1, 6, 2, 2, 1, 1, 1, 1

/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Accuracy Score,▁
f1-score_avg,▁
precision_avg,▁
recall_avg,▁
Accuracy Score,0.97719
f1-score_avg,0.97164
precision_avg,0.96643
recall_avg,0.97719


SVM Evaluation (for all)
Confusion Matrix:
[[175   1   0   0   0   0   0]
 [  2 235   0   0   0   0   0]
 [  0   0  36   0   0   1   0]
 [  0   0   0   0   0   1   0]
 [  0   1   1   0  12   0   0]
 [  0   0   1   0   0  19   0]
 [  0   0   0   0   0   0  15]]

Classification Report:
{'1': {'precision': 0.9887005649717514, 'recall': 0.9943181818181818, 'f1-score': 0.9915014164305949, 'support': 176.0}, '2': {'precision': 0.9915611814345991, 'recall': 0.9915611814345991, 'f1-score': 0.9915611814345991, 'support': 237.0}, '3': {'precision': 0.9473684210526315, 'recall': 0.972972972972973, 'f1-score': 0.9599999999999999, 'support': 37.0}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '5': {'precision': 1.0, 'recall': 0.8571428571428571, 'f1-score': 0.923076923076923, 'support': 14.0}, '6': {'precision': 0.9047619047619048, 'recall': 0.95, 'f1-score': 0.9268292682926829, 'support': 20.0}, '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 15.0}, '

/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Accuracy Score,▁
f1-score_avg,▁
precision_avg,▁
recall_avg,▁
Accuracy Score,0.984
f1-score_avg,0.98297
precision_avg,0.98232
recall_avg,0.984


In [23]:
ensemble_predictions = []
seconds = time.time()

for i in range(len(X_test_scaled_df)):
    test_instance_df = X_test_scaled_df.iloc[i].to_frame().T
    if knn_predictions[i] == NORMAL_TARGET:
        ensemble_predictions.append((0, NORMAL_TARGET, y_test_df.iloc[i]))
        continue
    predicted_label = TARGET_DICT[knn_predictions[i]]
    # print('predicted_label', predicted_label)
    selected_features = feature_importances[predicted_label]
    # print('selected_features', selected_features)
    model = ensemble_models[predicted_label]
    
    # Select features for the ensemble
    test_instance_selected_df = test_instance_df[selected_features]
    ensemble_prediction = model.predict(test_instance_selected_df)
    print('ensemble prediction: ', ensemble_prediction, 'knn prediction: ', INV_TARGET_DICT[predicted_label], 'actual value: ', y_test_df.iloc[i])
    ensemble_predictions.append((ensemble_prediction, INV_TARGET_DICT[predicted_label], y_test_df.iloc[i]))

print("Total operation time: = ", time.time() - seconds, "seconds")

ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [0] knn prediction:  3 actual value:  3
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  5 actual value:  5
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  5 actual value:  5
ensemble prediction:  [0] knn prediction:  3 actual value:  3
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [0] knn prediction:  6 actual value:  6
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [1] knn prediction:  1 actual value:  1
ensemble prediction:  [0] knn prediction:  3 actual value:  3
ensemble

In [24]:
wandb.init(project="pipeline-kdd-dataset", name="anomalies-ensemble")

# Now evaluate ensemble predictions only for the anomalies detected by KNN
# Create a mask for test instances that KNN classified as anomalies
anomaly_mask = knn_predictions != NORMAL_TARGET  # Assuming normal_target is your normal class
print('anomaly_mask', len(anomaly_mask))

# Get true labels and predictions for anomalies
ensemble_predictions_actual = [t[1] for t in ensemble_predictions]
print(ensemble_predictions_actual)
ensemble_predictions_actual_df = pd.DataFrame(ensemble_predictions_actual, columns=[TARGET_COLUMN])
y_test_anomalies_df = y_test_df[anomaly_mask]
ensemble_predictions_anomalies_df = ensemble_predictions_actual_df[anomaly_mask]
print('Lengths: ', len(y_test_anomalies_df), len(ensemble_predictions_anomalies_df))

# Evaluate ensemble only on the anomalies
conf_matrix, class_report, acc_score = evaluate(y_test_anomalies_df, ensemble_predictions_anomalies_df, heading='Ensemble Evaluation (for anomalies)')
wandb_log(conf_matrix, class_report, acc_score)
wandb.finish()

# Evaluate ensemble on all test data
wandb.init(project="pipeline-kdd-dataset", name="overall-ensemble")
conf_matrix, class_report, acc_score = evaluate(y_test_df, ensemble_predictions_actual_df, heading='Ensemble Evaluation (for all)')
wandb_log(conf_matrix, class_report, acc_score)
wandb.finish()

anomaly_mask 500
[1, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 1, 5, 2, 2, 1, 2, 5, 3, 2, 1, 2, 2, 1, 6, 1, 1, 2, 1, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 7, 2, 1, 1, 6, 2, 1, 1, 7, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 2, 2, 1, 5, 1, 2, 3, 1, 3, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 3, 1, 2, 1, 1, 2, 1, 3, 3, 2, 2, 1, 1, 6, 6, 2, 2, 1, 2, 5, 6, 6, 1, 2, 1, 2, 1, 1, 1, 3, 1, 2, 3, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 5, 1, 1, 1, 2, 3, 1, 2, 2, 3, 1, 1, 1, 2, 2, 2, 1, 7, 2, 2, 1, 1, 2, 2, 1, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 3, 3, 2, 1, 1, 2, 7, 1, 1, 1, 1, 2, 2, 2, 3, 2, 5, 3, 3, 2, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 2, 2, 1, 1, 2, 5, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 6, 2, 1, 2, 7, 2, 1, 3, 5, 1, 2, 3, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 6, 1, 3, 2, 7, 1, 1, 2, 2, 2, 2, 2, 2, 7, 1, 1, 1, 1, 1, 2, 2, 5, 2, 2, 2, 1, 6, 2, 1, 1, 2, 7, 1, 2, 3, 2, 2, 2, 2, 2, 6, 1, 5, 2, 2, 1, 6, 2, 2, 1, 1, 1, 1

/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Accuracy Score,▁
f1-score_avg,▁
precision_avg,▁
recall_avg,▁
Accuracy Score,0.97719
f1-score_avg,0.97164
precision_avg,0.96643
recall_avg,0.97719


Ensemble Evaluation (for all)
Confusion Matrix:
[[175   1   0   0   0   0   0]
 [  2 235   0   0   0   0   0]
 [  0   0  36   0   0   1   0]
 [  0   0   0   0   0   1   0]
 [  0   1   1   0  12   0   0]
 [  0   0   1   0   0  19   0]
 [  0   0   0   0   0   0  15]]

Classification Report:
{'1': {'precision': 0.9887005649717514, 'recall': 0.9943181818181818, 'f1-score': 0.9915014164305949, 'support': 176.0}, '2': {'precision': 0.9915611814345991, 'recall': 0.9915611814345991, 'f1-score': 0.9915611814345991, 'support': 237.0}, '3': {'precision': 0.9473684210526315, 'recall': 0.972972972972973, 'f1-score': 0.9599999999999999, 'support': 37.0}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '5': {'precision': 1.0, 'recall': 0.8571428571428571, 'f1-score': 0.923076923076923, 'support': 14.0}, '6': {'precision': 0.9047619047619048, 'recall': 0.95, 'f1-score': 0.9268292682926829, 'support': 20.0}, '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 15.

/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Accuracy Score,▁
f1-score_avg,▁
precision_avg,▁
recall_avg,▁
Accuracy Score,0.984
f1-score_avg,0.98297
precision_avg,0.98232
recall_avg,0.984
